### 🛠️ 1. Initialize notebook variables

Configures everything that's needed for deployment. 

[ADD ANY SPECIAL INSTRUCTIONS]

**Modify entries under _1) User-defined parameters_ and _3) Define the APIs and their operations and policies_**.

In [ ]:
import utils
from apimtypes import *

# 1) User-defined parameters (change these as needed)
rg_location = 'eastus2'
index       = 1
deployment  = INFRASTRUCTURE.AFD_APIM_PE
tags        = ['tag1', 'tag2', '...']       # ENTER DESCRIPTIVE TAG(S)
api_prefix  = ''                            # OPTIONAL: ENTER A PREFIX FOR THE APIS TO REDUCE COLLISION POTENTIAL WITH OTHER SAMPLES

# 2) Define the APIs and their operations and policies

# API 1
# api1_get = GET_APIOperation('This is a GET for API 1')
# api1_post = POST_APIOperation('This is a POST for API 1')
# api1 = API('API1', 'API 1', '/api1', 'This is API 1', operations = [api1_get, api1_post], tags = tags)

# API n
# ...

# APIs Array
# apis: List[API] = [api1, apin]
apis: List[API] = []

utils.print_ok('Notebook initialized')

### 🚀 2. Create deployment using Bicep

Creates the bicep deployment into the previously-specified resource group. A bicep parameters file will be created prior to execution.

In [ ]:
# Build the bicep parameters
bicep_parameters = {
    'apis': {'value': [api.to_dict() for api in apis]}
}

# Create the deployment helper and deploy the sample
nb_helper = utils.NotebookHelper('_TEMPLATE', utils.get_infra_rg_name(deployment, index), rg_location, deployment, [INFRASTRUCTURE.SIMPLE_APIM], index = index)
output = nb_helper.deploy_sample(bicep_parameters)

if output.success:
    # Extract deployment outputs for testing
    afd_endpoint_url = output.get('fdeSecureUrl', 'Front Door Endpoint URL')    # may be deleted if Front Door is not part of a supported infrastructure
    apim_name        = output.get('apimServiceName', 'APIM Service Name')
    apim_gateway_url = output.get('apimResourceGatewayURL', 'APIM API Gateway URL')
    apim_apis        = output.getJson('apiOutputs', 'APIs')

    utils.print_ok('Deployment completed')
else:
    print("❌ Deployment failed!")
    raise SystemExit(1)

### ✅ 3. Verify API Request Success

Assert that the deployment was successful by making simple calls to APIM. 

❗️ If the infrastructure shields APIM and requires a different ingress (e.g. Azure Front Door), the request to the APIM gateway URl will fail by design. Obtain the Front Door endpoint hostname and try that instead.

In [ ]:
from apimrequests import ApimRequests
from apimtesting import ApimTesting

# Initialize testing framework
tests = ApimTesting("Template Sample Tests", '_TEMPLATE', nb_helper.deployment)

# Example API testing (uncomment and customize as needed)
# api_subscription_key = apim_apis[0]['subscriptionPrimaryKey']

# Check if the infrastructure uses Azure Front Door
# utils.print_message('Checking infrastructure endpoint...', blank_above = True)
# afd_endpoint_url = utils.get_frontdoor_url(nb_helper.deployment, nb_helper.rg_name)

# if afd_endpoint_url:
#     # Test via Azure Front Door
#     reqsAfd = ApimRequests(afd_endpoint_url, api_subscription_key)
#     output = reqsAfd.singleGet('/', msg = 'Calling API via Azure Front Door. Expect 200.')
#     tests.verify('expected_value' in output, True)
# else:
#     # Test via API Management directly
#     reqsApim = ApimRequests(apim_gateway_url, api_subscription_key)
#     output = reqsApim.singleGet('/', msg = 'Calling API via API Management Gateway.')
#     tests.verify('expected_value' in output, True)

# tests.print_summary()
utils.print_ok('All done!')